In [1]:
### polygonio_data_puller
#Forest Kirschbaum 2022_10_26

###most useful
#https://greyhoundanalytics.com/blog/scraping-crypto-price-data-from-polygonio/


# https://polygon.io/docs/stocks/get_v2_aggs_ticker__stocksticker__range__multiplier___timespan___from___to
# https://raw.githubusercontent.com/AdamGetbags/polygonData/main/historicalOptionDataPolygon.py
# https://github.com/pssolanki111/polygon
# https://polygon.readthedocs.io/en/latest/bulk_data_download_functions.html
# https://github.com/AdamGetbags/polygonData/blob/main/historicalOptionDataPolygon.py
# https://www.geeksforgeeks.org/python-pandas-series-tz_convert/
# https://www.geeksforgeeks.org/python-pandas-series-tz_convert/



#     polygon_api_key = "Pl_Qzj5E0MAlH06aC5UGaFapWpqzKOSB" #get your own free from polygon.io
polygon_api_key = "Pl_Qzj5E0MAlH06aC5UGaFapWpqzKOSB"

In [2]:
import requests
import pandas as pd

from datetime import date, datetime, timedelta
import time
import os
from polygon import RESTClient
import datetime as dt

In [38]:
# for stock asset = "AAPL"
asset = "SPY"
# for crypto comment out the above line and...
# uncomment the two following lines for cryptofor crypto such as:  BTCUSD, UNIUSD, ETHUSD
# asset = "BTCUSD"
#asset = "X:" + asset


In [ ]:
lookback_months = 25

In [39]:
# 2022_10_17_jupyter = <<your python project in jupyter notebook or selected editor>>
DIR = r'/home/guest144/2022_10_17_jupyter/'
# data/polygon_assets = <<the path starting in dir, then adding onto where you want to store your data csvs.
target_dir = os.path.join(DIR, 'data/polygon_assets')

In [40]:
def pull_1m_data(asset, start_time, end_time):


    polygon_rest_baseurl = "https://api.polygon.io/v2/"
    multiplier = 1
    timespan = "minute"
    limit = 50000
    # newest data at the bottom
    sort = "asc"
        
    request_url = f"{polygon_rest_baseurl}aggs/ticker/{asset}/range/{multiplier}/" +\
            f"{timespan}/{start_time}/{end_time}?adjusted=true&sort={sort}&" + \
            f"limit={limit}&apiKey={polygon_api_key}"

    data = requests.get(request_url).json()

    if "results" in data:
        return data["results"]
    else:
        raise Exception("error_on_pull_1m_data")

In [41]:
bars = []

for i in range(lookback_months):
    end_time = date.today() - i*timedelta(days = 28)
    start_time = date.today() - (i+1)*timedelta(days = 28)
    bars += pull_1m_data(asset, start_time, end_time)
    time.sleep(15)

In [42]:
df = pd.DataFrame(bars)
df["date"] = pd.to_datetime(df["t"], unit = "ms")
df =  df[["date","o","h","c","l","v"]]
df.columns = ["time","Open","High","Low","Close","Volume"]
df['time'] = pd.to_datetime(df['time'])
df.set_index('time', inplace=True)
#localize time replace US/Central with <<your timezone>>
df.index = df.index.tz_localize('UTC')
df.index = df.index.tz_convert('US/Central') # df.index = df.index.tz_convert('America/New_York')
df = df.sort_values("time")

datapath = os.path.join(target_dir, asset + '.csv')
df.to_csv(datapath)

print(df)

                             Open    High       Low     Close  Volume
time                                                                 
2020-11-25 03:00:00-06:00  363.26  363.27  363.2200  363.2200  5679.0
2020-11-25 03:01:00-06:00  363.27  363.27  363.2700  363.2700  1135.0
2020-11-25 03:02:00-06:00  363.22  363.22  363.1800  363.1800  2004.0
2020-11-25 03:03:00-06:00  363.31  363.36  363.3600  363.3100  2742.0
2020-11-25 03:05:00-06:00  363.40  363.40  363.4000  363.4000   259.0
...                           ...     ...       ...       ...     ...
2022-10-25 18:55:00-05:00  381.32  381.32  381.2401  381.2401  3840.0
2022-10-25 18:56:00-05:00  381.25  381.28  381.2500  381.2300  3780.0
2022-10-25 18:57:00-05:00  381.25  381.31  381.2999  381.2500  3564.0
2022-10-25 18:58:00-05:00  381.28  381.28  381.1900  381.1800  2202.0
2022-10-25 18:59:00-05:00  381.19  381.19  381.1500  381.1500  4273.0

[420417 rows x 5 columns]
